In [1]:
# Basics
import pandas as pd
import numpy as np
from datetime import datetime


# Visualization
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
from windrose import WindroseAxes


# Preprocessing 
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer

**Dataset description:**

1) Windfarm's data:
    - 1 dataset from 2013 to 2016; data recorded every 10 min.
    - 1 dataset from 2017 to January 2018; data recorded every 10 min.

2) Meteorological station data:
    - 1 dataset from 2013 to January 2018; data recorder every 3 hrs.



### 1) Load Windfarm's data 
    - 1.1 Preliminary analysis
    - 1.2 Concatenate windfarm dataframes

In [2]:
# Load windfarm's data:

# Dataset 1: windfarm's data between 2013 and 2016
df_13_16= pd.read_csv('la-haute-borne-data-2013-2016.csv', delimiter= ';')

# Dataset 2: windfarm's data between 2017 and 2018
df_17_18= pd.read_csv('la-haute-borne-data-2017-2020.csv', delimiter= ';')

FileNotFoundError: [Errno 2] File la-haute-borne-data-2013-2016.csv does not exist: 'la-haute-borne-data-2013-2016.csv'

**1.1 Preliminary analysis**

In [ ]:
# Head of df_13_16; 138 columns are displayed
pd.set_option('display.max_columns', None)
df_13_16.head()

In [ ]:
# Checking shapes of windfarm's dataframes:

print('Shape 2013-2016:', df_13_16.shape)
print('Shape 2017-2018:', df_17_18.shape)

In [ ]:
# Checking the number of wind turbines and number records/turbine:
df_13_16['Wind_turbine_name'].value_counts() # 4 turbines; 2013 - 2016

In [ ]:
# Checking the number of wind turbines and number records/turbine:
df_17_18['Wind_turbine_name'].value_counts() # 4 turbines 2017-2018

**1.2 Concat windfarm dataframes**

In [ ]:
frames = [df_13_16, df_17_18] 
df_turb = pd.concat(frames) # concat df_13_16 & df_17_18

print('Shape of windfarm dataframe:', df_turb.shape)  # result shape

In [ ]:
# Delete df_13_16 & df_17_18
del df_13_16
del df_17_18

In [ ]:
# Sort dataframe by datetime and reset index:
df_turb= df_turb.sort_values('Date_time')
df_turb= df_turb.reset_index()

### 2) Preprocessing 
    - 2.1 Deletion of features with high NaN %
    - 2.2 Feature cleansing
    - 2.3 Create 1 dataframe per turbine
    - 2.4 Visualization of windfarm's data

**2.1 Deletion of features with high NaN %**

In [ ]:
# Calculate % of NaN
nulls= df_turb.isnull().sum()/len(df_turb)*100

In [ ]:
# Display columns and NaN %
#pd.set_option('display.max_rows', None) # to display all rows
nulls

In [ ]:
# Append on a list features/columns with NaN ≥ 20%

columns_high_NaN= []   # list

for column, NaN in zip(df_turb.columns, nulls): # iterate through columns and % NaN
    if NaN >= 20:  
        columns_high_NaN.append(column) # appends column name if NaN >= 20

In [ ]:
# Drop columns with NaN ≥ 20% from list of column names 
df_turb= df_turb.drop(columns_high_NaN, axis=1)

**2.2 Feature cleansing**
        
    - The correlation matrix has been used to delete features that (1) show excessive           correlation between each others, (2) are too correlataed with P (power) and (3) their     correlation is too low with the P. 
       
    - This process has been carried out with previous understanding of the significance each from feature

In [ ]:
# Heatmap with correlation matrix from all windfarm's features
# -- See 'heatmap in Jupyter Notebooks folder
#sns.set(rc={'figure.figsize':(100,100)}) 
#sns.heatmap(df_turb.corr(), annot = True, cmap = "Blues")

In [ ]:
# Drop columns non-correlated with the target (P)
df_turb= df_turb.drop(['Rbt_std', 'Rs_std', 'Nu_std',
                     'Nf_std',   'Nf_min',  'Nf_max', 'Nf_avg',
                     'Va_std',   'Va_min',  'Va_max', 'Va_avg',
                     'Wa_std',   'Wa_min',  'Ya_avg', 'Ya_min',
                     'Ya_max',   'Gost_std','Git_std','Gb1t_std',
                     'Gb2t_std', 'Db2t_std','Db2t_min','Db2t_max',
                     'Db2t_avg', 'Db1t_std','Db1t_min','Db1t_max',
                     'Db1t_avg', 'Cosphi_std','Cosphi_min','Cosphi_max',
                     'Cosphi_avg','DCs_std', 'Rt_std', 'Rt_min', 
                     'Rt_max', 'Rt_avg', 'Ba_std'], axis = 1)

In [ ]:
# Drop columns that have excessive correlation (aprox 1 and -1) with another feature
df_turb= df_turb.drop(['Wa_max', 'Nu_max', 'Nu_min', 'Nu_avg', 'Ya_std',
                     'Ds_std', 'Rm_std', 'Rm_avg', 'Rm_max', 'Rm_min',
                     'DCs_avg', 'DCs_max', 'DCs_min', 'Ds_avg', 'Ds_max',
                     'Ds_min', 'Gb1t_max', 'Gb1t_min', 'Gb2t_min', 'Gb2t_max',
                     'Dst_max', 'S_max', 'S_min', 'S_std', 'S_avg', 'Cm_max',
                     'Cm_min', 'Cm_std', 'Cm_avg', 'Dst_std'], axis = 1)

**2.3 Create 1 dataframe per turbine**

    - goal: concat with meteorological data each generator's dataframe
    - fill missing values with interpolate()


In [ ]:
# Function to clean 'Date_time' format
def clean_date(string):
    return " ".join(string.split("+")[0].split("T")) 

In [ ]:
# Clean "Date_time"
df_turb["Date_time"] = df_turb["Date_time"].apply(clean_date)

In [ ]:
# Create a dictionary of dataframes with 'Wind_turbine_name' as keys
dict_frames = dict(tuple(df_turb.groupby('Wind_turbine_name')))

# Save each dataframe within a new variable name
df_turb11 = dict_frames['R80711']
df_turb21 = dict_frames['R80721']
df_turb36 = dict_frames['R80736']
df_turb90 = dict_frames['R80790']

In [ ]:
del dict_frames # del dictionary to save some space

**2.4 Visualization of windfarm's data**

In [ ]:
# Power vs Wind Speed. 4 turbines, each one with a different color
sns.scatterplot(data=df_turb, x="Ws_avg", y="P_avg", hue="Wind_turbine_name")
sns.set(font_scale= 3)
plt.title('Wind speed vs Power. 4 generators')
sns.set(rc={'figure.figsize':(10,10)})
plt.show()

In [ ]:
# Select values recorded when the machine is stopped but the anemometers still work
sel_rows=df_turb[(df_turb['P_avg']<=0) &  (df_turb['Ws_avg']> 4.6)].index # where P <= 0 and Ws_avg > 4.6

In [ ]:
# Delete selected rows from df_gen
df_turb = df_turb.drop(sel_rows, axis=0)

In [ ]:
# Plot 'P_Avg' (power) vs rotor speed(RS), reactive power (Q_avg), stator temperature (DST), gearbox bearing temperature
# (Gb1t), gearbox oil temperature (Gost), rotor bearing temperature (Rbt)
for i in [ 'Rs_avg','Q_avg', 'Dst_avg', 'Gb1t_avg', 'Gost_avg','Rbt_avg']:
    sns.set(font_scale=2)
    sns.scatterplot(df_turb[i] ,df_turb.P_avg, color="r")
    plt.title('Power vs'+' '+i)
    sns.set(rc={'figure.figsize':(10,10)})
    plt.show()

### 3) Meteorological Data (independent station): processing

In [ ]:
# Load csv with meteorological data
df_met = pd.read_csv('donnees-synop-essentielles-omm_TRUE.csv', delimiter= ';')

In [ ]:
# Drop static useless data, such as city name, department etc...
df_met=df_met.drop(['Nom',          
'ID OMM station','Altitude','Longitude','Latitude',
'communes (name)','communes (code)','EPCI (name)',
'EPCI (code)','department (name)','department (code)',
'region (name)','region (code)','mois_de_l_annee','Coordonnees'], axis=1)

In [ ]:
# Calculate % nulls per column
nulls_meteo= df_met.isnull().sum()/len(df_met)*100
for column, percentage in zip(df_met.columns, nulls_meteo):
    print(column, percentage)

In [ ]:
# Append on a list features/columns with NaN ≥ 20%

columns_high_NaN= []   # list

for column, NaN in zip(df_met.columns, nulls_meteo): # iterate through columns and % NaN
    if NaN >= 18:  
        columns_high_NaN.append(column) # appends column name if NaN >= 20

In [ ]:
df_met = df_met.drop(columns_high_NaN, axis=1)

In [ ]:
# Delete irrelevant or repetitive information (already present in windfarm's data)
df_met = df_met.drop(['Direction du vent moyen 10 mn', 'Vitesse du vent moyen 10 mn', 
                      'Vitesse du vent moyen 10 mn','Temps présent', 
                      'Periode de mesure de la rafale', 'Point de rosée', 'Type de tendance barométrique.1',
                      'Temps présent.1', 'Température (°C)', 'Température' ], axis=1)

In [ ]:
# Function: clean date time column / erase '+'
def clean_date(string):
    return " ".join(string.split("+")[0].split("T")) 

# Clean date
df_met["Date"] = df_met["Date"].apply(clean_date)

In [ ]:
# Sort values by date 
df_met=df_met.sort_values(by='Date')

In [ ]:
# After deleting several columns:
df_met.head()

### 4) Feature Engineering 

    - 4.1 Time variables creation
    - 4.2 Create the target: P_3h
    - 4.3 Merge turbines with meteo
    - 4.4 Interpolate missing meteorological records

**4.1 Time variables creation**

    - Calculate variables 6h, 3h, 1h before each time record --> rolling()
    - Calculate TI (turbulence intensity) 
   

In [ ]:
# To avoid default warning when applying .rolling()
pd.options.mode.chained_assignment = None  

In [ ]:
# Define function with rolling()
def rolling_function(dataframe, new_column, variable, win_size):
    dataframe[new_column]= dataframe[variable].rolling(win_size).mean()
    return dataframe[new_column]

In [ ]:
# Apply rolling_function 3 times (1h, 3h, 6h) to: each df (dataframes) for 10 features (variables)
dataframes = [df_turb11, df_turb21, df_turb36, df_turb90]
variables = ['Git_avg', 'Ot_avg', 'Yt_avg', 'Rs_avg', 'Gb2t_avg',
             'Gb1t_avg', 'Gost_avg', 'Wa_avg', 'Ba_avg', 'Rbt_avg']
windows= [7, 19, 37]
names_hours= ['1h', '3h', '6h']

for dataframe in dataframes:
    for variable in variables:
        for window, name  in zip(windows, names_hours): # iterates through nº rows and names of new features

            rolling_function(dataframe, variable + '_' + name, variable, int(window))           

In [ ]:
# Calculate Turbulence intensity for each turbine --> std.dev Wind speed / avg Wind speed
dataframes = [df_turb11, df_turb21, df_turb36, df_turb90]
for d in dataframes: 
    d["TI"] = d["Ws_std"]/ d["Ws_avg"]

**4.2 Create the target: P_3h**

In [ ]:
# Calculate target (P_3h) for each generator
# Apply rolling on reverse mode to calculate P_3h (it takes 19 following rows)
dataframes = [df_turb11, df_turb21, df_turb36, df_turb90]
for d in dataframes:
    d["P_3h"] = d["P_avg"].iloc[::-1].rolling(19).mean() 
    d.P_3h= d.P_3h.shift(periods=18) # Shift/push P_3h row 19 lines on each df

In [ ]:
df_turb11= df_turb11.dropna(axis=0, subset=['P_3h'])
df_turb21= df_turb21.dropna(axis=0, subset=['P_3h'])
df_turb36= df_turb36.dropna(axis=0, subset=['P_3h'])
df_turb90= df_turb90.dropna(axis=0, subset=['P_3h'])

In [ ]:
# Remove 'P_avg', 'P_min', 'P_max', 'P_std' to avoid overfitting; they were used to create our target(P_3h)
dataframes = [df_turb11, df_turb21, df_turb36, df_turb90]
for d in dataframes:
    del d['P_avg']
    del d['P_min']
    del d['P_max']
    del d['P_std']

In [ ]:
df_turb11.shape

In [ ]:
df_turb21.head()

**4.3 Merge turbines with meteo**

In [ ]:
# Create common column 'datetime' in all generator frames and df_met 
dataframes = [df_turb11, df_turb21, df_turb36, df_turb90]
for d in dataframes:
    d.insert(2, 'datetime', d.Date_time) # 4 turbines
df_met.insert(2, 'datetime', df_met.Date) # meteo 

In [ ]:
#Convert 'datetime' column to_datetime type
dataframes = [df_turb11, df_turb21, df_turb36, df_turb90, df_met]
for d in dataframes:
    d.datetime = pd.to_datetime(d.datetime)
    d.set_index("datetime",inplace=True) # Set 'datetime' column as index in all dataframes

In [ ]:
# Merge turbine's dataframes with df_met
df_turb11= pd.merge(df_met, df_turb11, how="outer", left_index=True, right_index=True)
df_turb21= pd.merge(df_met, df_turb21, how="outer", left_index=True, right_index=True)
df_turb36= pd.merge(df_met, df_turb36, how="outer", left_index=True, right_index=True)
df_turb90= pd.merge(df_met, df_turb90, how="outer", left_index=True, right_index=True)

In [ ]:
df_turb11.head(40)

**4.4 Interpolate missing meteorological records.**

In [ ]:
# List of frames and columns to interpolate
d_frame= [df_turb11, df_turb21, df_turb36, df_turb90]
c_columns= ["Pression au niveau mer", "Variation de pression en 3 heures", "Humidité", 
         "Visibilité horizontale", "Pression station", "Rafales sur une période", 
         "Précipitations dans la dernière heure", "Précipitations dans les 3 dernières heures"]

# Interpolation of quantitative columns from df_met
for d in d_frame:
    for c in c_columns:
        d[c]= d[c].interpolate()

### 5) Work with all data: turbines + meteo

    - 5.1 Concatenate all turbines with meteo
    - 5.2 Processing all data
    - 5.3 Create variables from months and years

**5.1 Concatenate all turbines with meteo**

In [ ]:
# Concatenate 3 dataframes without duplicates
df_4turb= pd.concat([df_turb11, df_turb21, df_turb36, df_turb90]).drop_duplicates() #.reset_index(drop=True)

In [ ]:
del [df_turb11, df_turb21, df_turb36, df_turb90, df_met] # save memory

In [ ]:
df_4turb.shape

**5.2 Processing all data**

In [ ]:
#pd.set_option('display.max_columns', None)
df_4turb.head()

In [ ]:
# Drop first 145 columns due to NaNs produced during rolling() function 
# 6h / turbine 
df_4turb= df_4turb.drop(df_4turb.index[range(0,144)])

In [ ]:
# Drop due to high amount of NaN
df_4turb= df_4turb.drop(["Date", 
                           "Type de tendance barométrique",
                           "Nébulosité  des nuages de l' étage inférieur",
                           "index"], axis=1)

In [ ]:
# Change names of columns from meteo
col_dict= {'Pression au niveau mer': 'pr_sl', 'Variation de pression en 3 heures': 'pr_3h', 
         'Humidité': 'Hum', 'Visibilité horizontale': 'Vis_hor', 'Pression station': 'pr_st',
         'Rafales sur une période': 'W_blast', 'Précipitations dans la dernière heure': 'Rain_1h',
         'Précipitations dans les 3 dernières heures': 'Rain_3h',
         'Hauteur de base 1': 'Hgt_base1'}

df_4turb.columns = [col_dict.get(x, x) for x in df_4turb.columns]

**5.3 Create variables from months and years**

In [ ]:
df_4turb.Date_time = pd.to_datetime(df_4turb.Date_time)
df_4turb['month']= df_4turb.Date_time.apply(lambda x : x.month)

In [ ]:
# 'year' will only be used for statistical purposes; not for the model 
df_4turb['year']= df_4turb.Date_time.apply(lambda x : x.year)

In [ ]:
# Dummies on 'month'
dummy_variable = pd.get_dummies(df_4turb['month'])

In [ ]:
# Concat df with dummies
df_4turb = pd.concat([df_4turb, dummy_variable], axis=1)

In [ ]:
del df_4turb['month']

In [ ]:
# Change names of columns from months
col_dict2= {1.0: 'Jan', 2.0: 'Feb', 
         3.0: 'Mar', 4.0: 'Apr', 5.0: 'May',
         6.0: 'Jun', 7.0: 'Jul',
        8.0: 'Aug', 9.0: 'Sept',
        10.0 : 'Oct' ,11.0: 'Nov', 
            12.0: 'Dec'}

df_4turb.columns = [col_dict2.get(x, x) for x in df_4turb.columns]

### 6) Correlation matrix between all features

In [ ]:
# List comprehension to move target ('P_3h') to the right of the dataframe
end_col = ['P_3h']
df_4turb = df_4turb[[c for c in df_4turb if c not in end_col] 
        + [c for c in end_col if c in df_4turb]]

### 7) Windrose graphs

In [ ]:
# Windrose plot with filled representation of wind speeds (frequency) and wind directions
# Yearly data 
list_years= [2013, 2014, 2015, 2016, 2017]
for y in list_years:
    year= y
    figure = plt.figure(figsize=(12,12))
    legend_position = 'upper left'
    ax = figure.add_subplot(221,projection='windrose')
    ax.contourf(df_4turb.loc[df_4turb['year']== year,'Wa_avg'], df_4turb.loc[df_4turb['year']==year,'Ws_avg'], 
                 bins = np.arange(2, 12, 2),cmap=cm.hot)
    ax.contour(df_4turb.loc[df_4turb['year']==year,'Wa_avg'], df_4turb.loc[df_4turb['year']==year,'Ws_avg'],
                colors='black',bins = np.arange(2, 12, 2))
    ax.set_legend(loc= legend_position, fontsize= 18)
    plt.title('Year:' +' '+ str(y), y=1.1, fontsize= 15)
    plt.show()

In [ ]:
# Filter all months separatedly from the 5 years of records
jan= df_4turb[df_4turb['Jan']==1]
feb= df_4turb[df_4turb['Feb']==1]
mar= df_4turb[df_4turb['Mar']==1]
apr= df_4turb[df_4turb['Apr']==1]
may= df_4turb[df_4turb['May']==1]
jun= df_4turb[df_4turb['Jun']==1]
jul= df_4turb[df_4turb['Jul']==1]
aug= df_4turb[df_4turb['Aug']==1]
sept= df_4turb[df_4turb['Sept']==1]
octb = df_4turb[df_4turb['Oct']==1]
nov= df_4turb[df_4turb['Nov']==1]
dec= df_4turb[df_4turb['Dec']==1]

In [ ]:
# Stacked histogram with wind directions and speeds
# Data separated by months from all the years that we have records
dfs_months = [jan, feb, mar, apr, may, jun, jul, aug, sept, octb, nov, dec]
labels =['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

for i,j in zip(dfs_months,labels):

    ax = WindroseAxes.from_ax()
    ax.bar(i['Wa_avg'], i['Ws_avg'], normed=True, opening=0.8, edgecolor='white',cmap=cm.hot)
    ax.legend(bbox_to_anchor=(1.02, 0))
    ax.set_title((j), y=1, fontsize= 15)

In [ ]:
# save some memory space
del [jan, feb, mar, apr, may, jun, jul, aug, sept, octb, nov, dec, df_4turb['year']]

### 8) Last processing before modelling 

    - 8.1 Check for infinites and substitute
    - 8.2 Create dataframe from a single turbine
    - 8.3 Remove outliers with IQR
    - 8.4 Extract the target and save it as CSV
    - 8.5 Standarize
    - 8.6 NaN imputation: KNNimputer
    - 8.7 Export data for modelling as CSV

**8.1 Check for infinites and substitute**

In [ ]:
# Substitute possible infinites (due to division by zero) for nan
df_4turb= df_4turb.replace(np.inf, np.nan)

**8.2 Create dataframe from a single turbine**

In [ ]:
# Exctract a dataframe with a single generator
Turb11= df_4turb[df_4turb['Wind_turbine_name']=='R80711']

In [ ]:
del Turb11['Wind_turbine_name']
del Turb11['Date_time']
del df_4turb['Wind_turbine_name']
del df_4turb['Date_time']

**8.3 Remove outliers with IQR**

In [ ]:
# Data seems to be very homogeneous; if we settle Q1= 0.18 & Q3= 0.95 we loose more than 95% od data
Q1= df_4turb.quantile(0.05)
Q3= df_4turb.quantile(0.95)
IQR= Q3-Q1 # IQR is the first quartile subtracted from the third quartile

In [ ]:
#Using IQR 
df_4turb = df_4turb[~((df_4turb < (Q1 - 1.5 * IQR)) |(df_4turb > (Q3 + 1.5 * IQR))).any(axis=1)]
df_4turb.shape

In [ ]:
# IQR for a single turbine
Q1= Turb11.quantile(0.05)
Q3= Turb11.quantile(0.95)
IQR= Q3-Q1 
Turb11 = Turb11[~((Turb11 < (Q1 - 1.5 * IQR)) |(Turb11 > (Q3 + 1.5 * IQR))).any(axis=1)]
Turb11.shape

**8.4 Extract the target and save it as CSV**

- 4 Turbines:

In [ ]:
# Save copy of our target 
P_3h= df_4turb['P_3h']

In [ ]:
P_3h= pd.DataFrame(P_3h)

In [ ]:
P_3h.isnull().sum()

In [ ]:
P_3h.to_csv('P_3h.csv', index=True)

In [ ]:
del df_4turb['P_3h']

- 1 Turbine

In [ ]:
P_3h_Turb11= Turb11['P_3h']

In [ ]:
# Convert y to dataframe
P_3h_Turb11= pd.DataFrame(P_3h_Turb11)

In [ ]:
P_3h_Turb11.to_csv('P_3h_Turb11.csv', index=True)

In [ ]:
del Turb11['P_3h']

**8.5 Standarize**

- 4 Turbines:

In [ ]:
# Standarize
scaler = StandardScaler()
scaled = scaler.fit_transform(df_4turb)

In [ ]:
df= pd.DataFrame(scaled, columns=df_4turb.columns)

- 1 Turbine:

In [ ]:
# Standarize
scaler_turb11 = StandardScaler()
scaled_turb11 = scaler_turb11.fit_transform(Turb11)

In [ ]:
df_Turb11= pd.DataFrame(scaled_turb11, columns=Turb11.columns)

**8.6 NaN imputation: KNNimputer**

- 4 Turbines

In [ ]:
# KNNImputer for Nans
imputer = KNNImputer(n_neighbors=3)
imputed = imputer.fit_transform(df)

In [ ]:
df= pd.DataFrame(imputed, columns=df_4turb.columns)

- 1 Turbine

In [ ]:
# KNNImputer for Nans
imputer_turb11 = KNNImputer(n_neighbors=3)
imputed_turb11 = imputer_turb11.fit_transform(df_Turb11)

In [ ]:
df_Turb11= pd.DataFrame(imputed_turb11, columns=Turb11.columns)

**8.7 Export data for modelling as CSV**

In [ ]:
# CSV with 4 turbine's data
df.to_csv('4Gen_Model.csv', index=True)
# CSV with 1 turbine's data
df_Turb11.to_csv('Turb11_Model.csv', index=True)

In [ ]:
del [df, df_Turb11]